In [1]:
![ ! -f requirements.txt ] && git clone https://github.com/anudaweerasinghe/small-coder.git && cd small-coder

fatal: destination path 'small-coder' already exists and is not an empty directory.


In [2]:
%%capture

! pip3 install -r small-coder/requirements.txt

In [3]:
# setup human eval and mbpp dataset
from datasets import load_dataset
humaneval = load_dataset("openai_humaneval", split="test")
mbpp = load_dataset("mbpp", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def mbppToFnHeaderAndDocString(mbpp_datum):
  ref_implementation = mbpp_datum['code']
  prompt = mbpp_datum['text']

  fn_header = ref_implementation.split('\n', 1)[0]

  return f'{fn_header}\n    """\n    {prompt}\n    """'


In [5]:
def getHumanEvalTestsList(humaneval_datum):
  check_fn = humaneval_datum['test']

  lines = [line.strip() for line in check_fn.split('\n')]

  tests = list(filter(lambda string: string.startswith('assert'), lines))

  return tests

In [6]:
# View one example of humaneval and mbpp
print('=================HumanEval=================')
print(humaneval[0]['prompt'])
print(getHumanEvalTestsList(humaneval[0]))

print('===================MBPP===================')
print(mbppToFnHeaderAndDocString(mbpp[0]))
print(mbpp[0]['test_list'])

=================HumanEval=================
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True', 'assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True', 'assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False', 'assert candidate([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) == True', 'assert candidate([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) == False']
===================MBPP===================
def remove_Occ(s,ch): 
    """
    Write a python function to remove first and last occurrence of a given character from th

In [7]:
#initialize deepsek-coder-1.3b
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# helper to generate code for agiven docstring
def generateCode(docstring):
  message = [
    {'role': 'user', 'content': 'Complete the following code. Do not generate test cases.'},
    { 'role': 'user', 'content': docstring  }
  ]

  inputs = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors="pt").to(model.device)
  # tokenizer.eos_token_id is the id of <|EOT|> token
  outputs = model.generate(inputs, max_new_tokens=128000, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)

  return tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

In [9]:
humanevalExampleResult = generateCode(humaneval[0]['prompt'])
mbppExampleResult = generateCode(mbppToFnHeaderAndDocString(mbpp[0]))

print('=================HumanEval=================')
print(humanevalExampleResult)

print('===================MBPP===================')
print(mbppExampleResult)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to o

=================HumanEval=================
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False

===================MBPP===================
def remove_Occ(s,ch): 
    """
    Write a python function to remove first and last occurrence of a given character from the string.
    """
    if ch in s:
        s = s.replace(ch, '', 1)
        s = s.replace(ch, '', 1)
    return s



In [10]:
# allow running code for evaluation - THIS IS DANGEROUS
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [11]:
from evaluate import load
code_eval = load("code_eval")

In [12]:
def evaluateCode(code, tests):
  print(code)
  print(tests)
  candidates = [[code]]

  pass_at_k, results = code_eval.compute(references=tests, predictions=candidates, k=[1])

  return pass_at_k, results

In [24]:
test_cases = [getHumanEvalTestsList(humaneval[0])[0]]
candidates = [[humanevalExampleResult]]

print(test_cases, candidates)
pass_at_k, results = code_eval.compute(references=test_cases, predictions=candidates, k=[1])
print(results)

['assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True'] [['from typing import List\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for i in range(len(numbers)):\n        for j in range(i + 1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False\n']]


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


defaultdict(<class 'list'>, {0: [(0, {'task_id': 0, 'passed': False, 'result': "failed: name 'candidate' is not defined", 'completion_id': 0})]})
